# 代码测试开始

准备教科书

In [5]:
import re
file = '麻醉教材/麻醉学高级教程/2024_07_10_373f31b88d2bf633007bg/2024_07_10_373f31b88d2bf633007bg.tex'

filebase = "麻醉学高级教程"

def split_tex_by_section(filename):
    with open(file, 'r') as f:
        tex_content = f.read()
    sections = re.split('\\\\section\*', tex_content)
    # sections = ['\section*' + sec for sec in sections if sec]
    sections = [section.replace('\n', ' ').replace('\\', ' ') for section in sections]
    return sections
def make_datasets(sections):
    datasets = []
    current_set = ""
    current_len = 0
    size = 4000
    for section in sections:
        section_len = len(section)
        
        if section_len + current_len < size:
            current_set += section
            current_len += section_len
        else:
            if section_len >= size:
                # 将超长的section拆分为多个部分
                while len(section) >= size:
                    datasets.append(section[:size - 2])
                    section = section[size - 2:] # 允许 100 个重合
                current_set = section
                current_len = len(section)
            else:
                datasets.append(current_set)
                current_set = section
                current_len = section_len
    if current_set:
        datasets.append(current_set)
    return datasets
sections = split_tex_by_section(file)
print(len(sections))
datasets = make_datasets(sections)
print(len(datasets))
# for dataset in datasets:
#     print(len(dataset))

2078
428


In [6]:
prefix_prompt = """
以下内容来自麻醉学高级教程部分PDF通过OCR得到的文字片段。 教程的内容可以被用于医学模型Q&A的训练。但是没有被整理和格式化。文本片段的内容不全是有价值的数据，而且有噪声。你需要自行判断什么样的数据可以被拿来作为数据集。


给定内容，提取其中有医学价值的信息，组成问答对，请你帮我提取格式化成正确的json形式：

[{"query": 问题, "answer": 答案},
{"query": 问题, "answer": 答案},
...
]
注意，不要返回其他内容。答案至少需要200字。重要的数据不能丢失。

还需要保证生成结果的json格式完整。

内容如下：

"""

In [4]:
import time 
class LoginLogoutManager():
    def __init__(self,):
        self.userconfig = [
            {"username": 'ziweicen@icloud.com', "password":'openai111czw'},
            {"username":'zhiwei.zhiwei@u.nus.edu',"password":'timfy0-baSwiw-zyxqet'},
            {"username":'chenzhw57@gmail.com','password':'gongxue@.5notlen'}
        ]
        self.idx = 0

    def wait_until_present(self, selector):
        start_time = time.time()
        timeout = 20
        while time.time() - start_time < timeout:
            if t.present(selector):
                return True
            time.sleep(1)  # 等待1秒后再次检查
        t.close()
        raise TimeoutError(f"超过等待时间 {timeout} 秒，无法找到元素 {selector}")

        # while(t.present(id) != True):
        #     t.wait(1)
        # return
    
    def log_out_chatgpt(self, ):
        try:
            t.close()
        except Exception as e:
            print(e)
        t.init(visual_automation = True)
        t.url('http://chatgpt.com')
        self.wait_until_present("//button[contains(@data-testid, 'profile-button')]")
        if not t.present("//button[contains(@data-testid, 'profile-button')]"):
            print("找不到profile button")
            return 
        t.click("//button[contains(@data-testid, 'profile-button')]")
        t.click("注销")
        t.close()
        return
    
    def login(self, username, password):
        try:
            t.close()
        except Exception as e:
            print(e)
        t.init(visual_automation=True)
        t.url('http://chatgpt.com')

        self.wait_until_present("//button[contains(@data-testid,'welcome-login-button')]")
        if t.present("//button[contains(@data-testid,'welcome-login-button')]"):
            t.click("//button[contains(@data-testid,'welcome-login-button')]")

        self.wait_until_present("//input[contains(@class, 'email-input')]")
        if t.count("//input[contains(@class, 'email-input')]") == 1:
            t.type("//input[contains(@class, 'email-input')]", username)
        if t.count("继续") == 1:
            t.click("继续")
        self.wait_until_present("//input[contains(@name,'password')]")
        if t.count("//input[contains(@name,'password')]") == 1:
            t.type("//input[contains(@name,'password')]", password)
        self.wait_until_present("继续")
        if t.count("继续") == 1:
            t.click("继续")
        self.wait_until_present("//textarea[@id='prompt-textarea']")
        print(f"log in success username: {username}")

    def first_login(self,):
        self.login(self.userconfig[self.idx]['username'],self.userconfig[self.idx]['password'])
        
    def refresh(self,):
        self.idx = (self.idx + 1)%len(self.user_config)
        self.log_out_chatgpt()
        self.login(self.userconfig[self.idx]['username'],self.userconfig[self.idx]['password'])

In [7]:
import json
import tagui as t
import time 
import os 
import pyperclip as clip
import random
import sys 
if sys.platform == 'darwin':
    MODIFIER = '[cmd]'
elif sys.platform == 'linux':
    MODIFIER = '[ctrl]'
else:
    raise NotImplementedError(f"{sys.platform} not implmented!")


def write_json(filename, content):
    # 不保证json格式完整
    with open(filename, 'w') as file:
        file.write(content)



def generate_dataset(text):
    login_logout_manager = LoginLogoutManager()
    # login_logout_manager.first_login()
    t.init(visual_automation = True)
    t.url('https://www.chatgpt.com')

    def wait_until_present(selector, timeout=15):
        start_time = time.time()
        while time.time() - start_time < timeout:
            if t.present(selector):
                return True
            time.sleep(1)  # 等待1秒后再次检查
        t.close()
        raise TimeoutError(f"超过等待时间 {timeout} 秒，无法找到元素 {selector}")

    def wait_til_no_stop_or_continue_button():
        while True:
            # 如果存在“继续生成”按钮
            if t.present("继续生成"):
                t.click("继续生成")
                while not t.present("//button[contains(@data-testid, 'stop-button')]"):
                    t.wait(1)  # 等待直到“停止”按钮出现
                t.wait(2)  # 确保“停止”按钮完全加载
            else:
                # 等待直到“停止”按钮消失
                while t.present("//button[contains(@data-testid, 'stop-button')]"):
                    t.wait(1)  # 每秒检查一次
                t.wait(2)  # 确保“停止”按钮完全消失

            # 检查是否还有“继续生成”按钮
            if not t.present("继续生成"):
                break  # 没有“继续生成”按钮，退出循环
        return


    try:
        clip.copy(text)
        wait_until_present("//textarea[@id='prompt-textarea']")
        t.keyboard('[esc]')
        res = t.click("//textarea[@id='prompt-textarea']")
        print(f"点击状态: {res}")
        res = t.keyboard(f"{MODIFIER}v")
        print(f"粘贴状态: {res}")
        wait_until_present("//button[@data-testid='send-button']")
        res = t.click("//button[@data-testid='send-button']")
        print(f"点击发送状态：{res}")
  
        wait_til_no_stop_or_continue_button() 
        if t.present("//div[contains(@class, 'text-sm text-token-text-error')]"):
            print("text-sm text-token-text-error")
            login_logout_manager.refresh()
            return_string = clip.paste()
            return return_string
        if t.present("复制代码"):
            copy_status = t.click("复制代码")
            print(f"复制状态: {copy_status}")
        # t.close()
        else:
            print(f"找不到： 复制代码 按钮")
    except Exception as e:
        print(f"捕捉到错误：{e}")
        t.close()
        return clip.paste()

    t.close()
    result_str = clip.paste()
    return result_str

In [8]:
sections = datasets
import random


while True:
    # for i in range(len(sections)):
    random_indices = random.sample(range(len(sections)), len(sections))
    for i in random_indices:

        filename = f"gptqa/{filebase}_section{i}_4000_prompt2.json"
        if not os.path.exists(filename):
            ret = generate_dataset(prefix_prompt + sections[i])
            if ret == prefix_prompt + sections[i]:
                print(f"section {i} failed!")
            else:
                write_json(filename=filename, content=ret)
                print(f"i = {i}/{len(sections)}, successfully write {filename}.")

点击状态: True
粘贴状态: True
点击发送状态：True
复制状态: True
i = 258/428, successfully write gptqa/麻醉学高级教程_section258_4000_prompt2.json.
点击状态: True
粘贴状态: True
点击发送状态：True
复制状态: True
i = 236/428, successfully write gptqa/麻醉学高级教程_section236_4000_prompt2.json.
点击状态: True
粘贴状态: True
点击发送状态：True
复制状态: True
i = 387/428, successfully write gptqa/麻醉学高级教程_section387_4000_prompt2.json.
捕捉到错误：超过等待时间 15 秒，无法找到元素 //textarea[@id='prompt-textarea']
[RPA][ERROR] - use init() before using close()
section 406 failed!
点击状态: True
粘贴状态: True
点击发送状态：True
复制状态: True
i = 224/428, successfully write gptqa/麻醉学高级教程_section224_4000_prompt2.json.
点击状态: True
粘贴状态: True
点击发送状态：True
复制状态: True
i = 266/428, successfully write gptqa/麻醉学高级教程_section266_4000_prompt2.json.


In [9]:
len(datasets)

428

In [1]:
import tagui as t 
t.init(visual_automation = True)
t.url('http://chatgpt.com')

True

找不到profile button


In [8]:
t.count("//img[contains(@alt,'User')]")

2

In [11]:
t.click("//button[contains(@data-testid, 'profile-button')]")
t.click("注销")
t.close()

True

login logout test

In [1]:
import tagui as t
user_config = [
    {"username": 'ziweicen@icloud.com', "password":'openai111czw'},
    {"username":'zhiwei.zhiwei@u.nus.edu',"password":'timfy0-baSwiw-zyxqet'},
    {"username":'chenzhw57@gmail.com','password':'gongxue@.5notlen'}
]

def wait_until_present(id):
    while(t.present(id) != True):
        t.wait(1)
    return
def log_out_chatgpt():
    try:
        t.close()
    except Exception as e:
        print(e)
    t.init(visual_automation = True)
    t.url('http://chatgpt.com')
    wait_until_present("//button[contains(@data-testid, 'profile-button')]")
    if not t.present("//button[contains(@data-testid, 'profile-button')]"):
        print("找不到profile button")
        return 
    t.click("//button[contains(@data-testid, 'profile-button')]")
    t.click("注销")
    t.close()
    return
def login(username = 'ziweicen@icloud.com', password = 'openai111czw'):
    try:
        t.close()
    except Exception as e:
        print(e)
    t.init(visual_automation=True)
    t.url('http://chatgpt.com')

    wait_until_present("//button[contains(@data-testid,'welcome-login-button')]")
    if t.present("//button[contains(@data-testid,'welcome-login-button')]"):
        t.click("//button[contains(@data-testid,'welcome-login-button')]")

    wait_until_present("//input[contains(@class, 'email-input')]")
    if t.count("//input[contains(@class, 'email-input')]") == 1:
        t.type("//input[contains(@class, 'email-input')]", username)
    if t.count("继续") == 1:
        t.click("继续")
    wait_until_present("//input[contains(@name,'password')]")
    if t.count("//input[contains(@name,'password')]") == 1:
        t.type("//input[contains(@name,'password')]", password)
    wait_until_present("继续")
    if t.count("继续") == 1:
        t.click("继续")
    wait_until_present("//textarea[@id='prompt-textarea']")
    print(f"log in success username: {username}")

# 使用方法
idx = 0
while True:
    login(user_config[idx]['username'],user_config[idx]['password'])
    log_out_chatgpt()
    idx = (idx + 1)%len(user_config)

